In [2]:
import sys

In [3]:
import pandas as pd
import librosa
import torch
import onnxruntime as ort
from glob import glob
from code_base.datasets import WaveAllFileDataset
from code_base.inefernce import BirdsInference
from code_base.utils import load_json
%matplotlib inline

In [4]:
EXP_NAME = "InferenceClass-v1"
TRAIN_PERIOD = 5
CONFIG = {
    # Main
    "run_validation": False,
    "run_test": True,
    "use_sigmoid": False,
    "folds":[0],
    "test_data_root": "./testaudio/*.ogg",
    "label_map_data_path":'./json/bird2int_2023.json',
    "lookback":None,
    "lookahead":None,
    "segment_len":5,
    "step": None,
    "late_normalize": True,
    "exp_name":EXP_NAME,

}

if CONFIG.get("use_sed_mode", False):
    assert CONFIG["step"] is not None
else:
    assert CONFIG["step"] is None
    
if "folds" not in CONFIG:
    CONFIG["folds"] = list(range(CONFIG["n_folds"]))

In [5]:
bird2id = load_json(CONFIG["label_map_data_path"])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9336\203894239.py:6: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  "duration_s": [librosa.get_duration(filename=el) for el in test_au_pathes]


In [6]:
if CONFIG["run_test"]:
    test_au_pathes = glob(CONFIG["test_data_root"])

    test_df = pd.DataFrame({
        "filename": test_au_pathes,
        "duration_s": [librosa.get_duration(filename=el) for el in test_au_pathes]
    })

In [7]:
if CONFIG["run_test"]:
    ds_config_test = {
       "root": "",
       "label_str2int_mapping_path": CONFIG["label_map_data_path"],
       "n_cores": 64,
       "use_audio_cache": True,
       "test_mode": True,
       "segment_len": CONFIG["segment_len"],
       "lookback":CONFIG["lookback"],
       "lookahead":CONFIG["lookahead"],
        "sample_id": None,
        "late_normalize": CONFIG["late_normalize"],
        "step": CONFIG["step"],
        "validate_sr": 32_000
    }
loader_config = {
    "batch_size": 4,
    "drop_last": False,
    "shuffle": False,
    "num_workers": 0,
}

secondary_labels is not found in df. Maybe test or nocall mode


In [8]:
if CONFIG["run_test"]:
    ds_test = WaveAllFileDataset(df=test_df, **ds_config_test)

In [9]:
if CONFIG["run_test"]:
    loader_test = torch.utils.data.DataLoader(
        ds_test,
        **loader_config,
    )

In [10]:
model = ort.InferenceSession("./model/model.onnx")

In [11]:
inference_class = BirdsInference(
    device="cpu",
    verbose_tqdm=True,
    use_sigmoid=CONFIG["use_sigmoid"],
)

  0%|          | 0/3 [00:00<?, ?it/s]

Loading ./testaudio\XC128013.ogg to audio cache
Loading took 0.25699901580810547 seconds


100%|██████████| 3/3 [00:00<00:00,  3.06it/s]


In [12]:
test_preds, test_preds_long, test_dfidx, test_end = inference_class.predict_test_loader(
    nn_models=model,
    data_loader=loader_test,
    is_onnx_model=True
)

In [13]:
go_df = pd.DataFrame(test_preds);

result = []

for index, row in go_df.iterrows():
    max_index = row.idxmax()
    result.append(max_index)

index_counts = pd.Series(result).value_counts()

most_frequent_index = index_counts.idxmax()

In [14]:
import json

json_file_path = './json/bird2int_2023.json'

with open(json_file_path, 'r') as file:
    data_dict = json.load(file)

list_of_key = list(data_dict.keys())
list_of_value = list(data_dict.values())

position = list_of_value.index(most_frequent_index)
bird_id_string = list_of_key[position]

In [15]:
Bird_info = pd.read_csv('./eBird_Taxonomy_v2021.csv')
Bird_info = pd.DataFrame(Bird_info)

Turdus tephronotus


In [ ]:
found_row = Bird_info[Bird_info['SPECIES_CODE'] == bird_id_string]
result = found_row['SCI_NAME'].values[0]
print(result)